Work in Progress

**Installing** dependency

we are keeping for now all the file of ALBERT in transformers.jl (it will Ultimately be added in TextAnalysis.jl with Transformers.jl as its dependency) we are using tokenize and Wordpiece(sentencepiece) from Albert which is based on Transformers.jl

In [1]:
using Transformers

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/iamtejas/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


Here we are taking 3 sentence as sample

In [24]:
sample1 = "God is Great! I won a lottery."
sample2 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample3 = "She had the job she had planned for the last three years."
sample = [sample1,sample2,sample3]


3-element Array{String,1}:
 "God is Great! I won a lottery."                                                                                                                                 
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."                                                                                                      

ALBERT uses SentencePiece for word segmentation and Sentence segementation.To get better performance we first tokenise the Sentences by space(using Tokenise function of Albert) and then use SentencePiece for Word Segmentation and Encoding.

we first replace " "(space) with ' _' [space(U+2581)]

To decode split sentence. SentencePiece replace space with U+2581 and Store word in .vocab file with U+2581 as prefix .To tackle this we are using little trick as below.

In [ ]:
We will replace it with better implementation of SentencePiece 

In [3]:
sample[1] = replace(sample[1]," "=> " ▁")
sample[2] = replace(sample[2]," "=> " ▁")
sample[3] = replace(sample[3]," "=> " ▁")


"She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."

We Can use WordPiece for Word Segmentation with ALBERT vocabulary as follow. 
working on the APIs for Wordpiece

In [4]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [6]:
using Transformers.BidirectionalEncoder

In [7]:
tokens = tokenise.(sample, Val(true))


3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [8]:
vocab = readlines("/home/iamtejas/Downloads/albert_xlarge_v1/albert_xlarge/30k-clean.vocab")
vocabnew = split.(vocab , "\t")
vo = [vocabnew[1][1]]


1-element Array{SubString{String},1}:
 "<pad>"

We don't need Score of SentencePiece unigram model for Wordpiece.

In [9]:
for i in 1:30000
    vocab1 = vocabnew[i][1]
    push!(vo,vocab1)
end
vocab1 = convert(Array{String,1},vo)

30001-element Array{String,1}:
 "<pad>"        
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [17]:
vocab1 = vocab1[2:30001]

30000-element Array{String,1}:
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 "€"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [18]:
wp = WordPiece(vocab1,"<unk>"; max_char = 200)

WordPiece(vocab_size=30000, unk=<unk>, max_char=200)

In [19]:
tokensnew = wp.(tokens)

3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [20]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [21]:
vocabalbert = Transformers.Basic.Vocabulary(wp.vocab,wp.vocab[wp.unk_idx])

Vocabulary(30000, unk=<unk>)

here we have indices form sample tokens.


In [23]:
indices = vocabalbert(tokensnew)

30×3 Array{Int64,2}:
  5475    822  1080
    26     66    42
   375     67    15
   188  13528  1206
    32     20    40
   231     15    42
    22    133  2036
 17519    819    27
    10     25    15
     2     42   237
     2     75   133
     2    881   123
     2     21    10
     ⋮             
     2     16     2
     2     33     2
     2     24     2
     2  22569     2
     2     31     2
     2     60     2
     2     84     2
     2    234     2
     2     22     2
     2  22740     2
     2  20600     2
     2     10     2

since we only take one sentence as input

In [25]:
seg_indices = ones(Int, size(indices)...)

30×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

Next we need to turn those indices into embeddings.These can be done by loading embedding in Embed 